In [13]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [14]:
train = pd.read_csv('data_train.csv')
test = pd.read_csv('data_test.csv')
train = train.drop(['Unnamed: 0'],axis=1)
test = test.drop(['Unnamed: 0'],axis=1)

In [15]:
train.vmean = train.vmean.loc[train.vmean<90]
train.vmax = train.vmean.loc[train.vmax<90]
train.vmin = train.vmean.loc[train.vmin<90]

In [16]:
test['vmax']=test['vmax'].replace( np.NaN , test['vmax'].mean() )
test['vmin']=test['vmin'].replace( np.NaN , test['vmin'].mean() )
test['vmean']=test['vmean'].replace( np.NaN , test['vmean'].mean() ) 

train['vmax']=train['vmax'].replace( np.NaN , train['vmax'].mean() )
train['vmin']=train['vmin'].replace( np.NaN , train['vmin'].mean() )
train['vmean']=train['vmean'].replace( np.NaN , train['vmean'].mean() )

In [17]:
train.vmean = np.where((train.time_entry==train.time_exit) | ((train.x_entry==train.x_exit) & (train.y_entry==train.y_exit)) | (train.vmean<0) , 0, train.vmean)
train.vmax = np.where((train.time_entry==train.time_exit) | ((train.x_entry==train.x_exit) & (train.y_entry==train.y_exit)) | (train.vmax<0) , 0, train.vmax)
train.vmin = np.where((train.time_entry==train.time_exit) | ((train.x_entry==train.x_exit) & (train.y_entry==train.y_exit)) | (train.vmin<0), 0, train.vmin)

test.vmean = np.where((test.time_entry==test.time_exit) | ((test.x_entry==test.x_exit) & (test.y_entry==test.y_exit)) | (test.vmean<0) , 0, test.vmean)
test.vmax = np.where((test.time_entry==test.time_exit) | ((test.x_entry==test.x_exit) & (test.y_entry==test.y_exit)) | (test.vmax<0) , 0, test.vmax)
test.vmin = np.where((test.time_entry==test.time_exit) | ((test.x_entry==test.x_exit) & (test.y_entry==test.y_exit)) | (test.vmin<0), 0, test.vmin)

In [18]:
def convertisseurSeconde(time):
    sec = 3600*int(time[0:2]) + 60*int(time[3:5]) + int(time[6:8])                                                                                                                                  
    return (sec)

convertvec = np.vectorize(convertisseurSeconde)

In [19]:
train.time_entry = convertvec(train.time_entry)
train.time_exit = convertvec(train.time_exit)

test.time_entry = convertvec(test.time_entry)
test.time_exit = convertvec(test.time_exit)

In [20]:
train['city_center']=0
train['city_center'][(train.x_exit>=3750901.5068) & (train.x_exit<=3770901.5068)&(train.y_exit>=(-19268905.6133)) & (train.y_exit<=(-19208905.6133))]=1

C:\Users\gouzm\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
training = train.drop(['hash','trajectory_id','x_exit','y_exit'],axis=1)
#model = XGBClassifier()
y_train = training.city_center
x_train = training.drop(['city_center'],axis=1)
#model.fit(x_train,y_train)

In [22]:
X = x_train
y = y_train
param_grid = {
    'num_boost_round': [100],
    'eta': [0.1],
    'max_depth': [5,6,7],
    'min_child_weight': [0.5,1,2]
}

i=1
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)

for train_index,test_index in kf.split(X,y):
    print('\n{} of kfold {}'.format(i,kf.n_splits))
    xtr,xvl = X.loc[train_index],X.loc[test_index]
    ytr,yvl = y[train_index],y[test_index]
    model = GridSearchCV(XGBClassifier(), param_grid, cv=5, scoring= 'f1',iid=True)
    print ('---')
    model.fit(xtr, ytr)
    print (model.best_params_)
    pred=model.predict(xvl)
    print('accuracy_score',accuracy_score(yvl,pred))
    i+=1


1 of kfold 5
---


KeyboardInterrupt: 

In [ ]:
#xgb.plot_importance(model, max_num_features = 15)
#feat_importancesXGB = pd.Series(model.feature_importances_)
#feat_importancesXGB.nlargest(10).plot(kind='barh')

In [ ]:
x_test = test.loc[test['x_exit'].isnull()]
trajectory_id = x_test.trajectory_id
x_test = x_test.drop(['hash','trajectory_id','x_exit', 'y_exit'],axis=1)
prediction = model.predict(x_test)

trajectory_id = trajectory_id.values
sub = pd.DataFrame()
sub['id']=trajectory_id
sub['target']=prediction
sub.to_csv("XGBTresEntraine.csv",index =  False)

In [23]:
model.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'